In [1]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
import os
import tempfile
import numpy as np
import pandas as pd
import csv
import string
import json
import tensorflow as tf
import pickle

#se importa la colección de documentos 
from gensim import corpora, models, similarities
#from gensim.models.phrases import Phrases, Phraser
from gensim.models import Doc2Vec
from gensim.models.doc2vec import TaggedDocument

import nltk
from nltk.util import ngrams
from nltk.corpus import stopwords

from sklearn.cluster import KMeans
from sklearn import metrics
from sklearn.decomposition import PCA

import matplotlib.pyplot as plt

C:\Users\Victor\Anaconda3\envs\tensorflowCPU\lib\site-packages\gensim\utils.py:1209: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
2019-08-20 15:31:12,857 : INFO : 'pattern' package not found; tag filters are not available for English


In [2]:
#lectura de un archivo json
def open_file(fname):
    with open(fname) as jsonfile:
            resumes = json.load(jsonfile)
    return resumes

In [3]:
#creación de un arreglo [actualjob + jobsexperience]
def create_arr_by_job(resumes):
    df=pd.DataFrame(resumes)
    job_desc=[]
    for cv in df.Resume:
        puesto=cv["job"].lower()
        desc=cv["summary"].lower()
        if len(desc)>0:
            job_desc.append([puesto,desc])

    print('Total de CV:{}'.format(len(job_desc)))
    i=0
    for cv in df.Resume:
        puesto=''
        desc=''
        for ex in cv["experience"]:
            try:
                puesto=ex["e_job"].lower()
            except KeyError:
                print('Campo job vacio')
            try:
                desc=ex["e_activities"].lower()
            except KeyError:
                print('Campo activities vacio')
            if len(desc)>0:
                job_desc.append([puesto,desc])
                i=i+1
    print('Total de puestos por experiencia:{}'.format(i))        
    print('Total de puestos extraidos:{}'.format(len(job_desc)))
    return job_desc

#etiquetado de puestos
def label_jobs(temp_file,job_desc):
    f= open(temp_file,"w+")
    jobs_labeled=[]
    documents=[]

    #stopwords
    stoplist = list(set(stopwords.words('english')))
    stoplist.extend(["–","/","&",".","$","(",")","!","?",";",":","'",","])

    tnzr = nltk.tokenize
    for job in job_desc:
        #Etiquetado de puestos
        if job[0].find("vice president")!= -1 or job[0].find("delivery manager")!= -1 or job[0].find(" vp ")!= -1 or job[0].find("svp ")!= -1:
            f.write("{} \t|\t vice president or equivalent\n".format(job[0]))
            label="vice president or equivalent"
        else :
            if job[0].find("president")!= -1:
                    f.write("{} \t|\t president or equivalent\n".format(job[0])) 
                    label="president or equivalent"
            else:            
                if job[0].find("director")!= -1:
                    f.write("{} \t|\t director or equivalent\n".format(job[0]))
                    label="director or equivalent"
                else:
                    if job[0].replace("sr","senior").find("senior project manager")!= -1 or (job[0].replace("sr","senior").find("senior")!= -1
                            and job[0].find("project manager")!= -1) or ( job[0].replace("programme","program").find("program")!= -1 and job[0].find("manager")!= -1):
                            f.write("{} \t|\t senior project manager or equivalent\n".format(job[0].replace("|",",")))
                            label="senior project manager or equivalent"
                    else:
                        if job[0].find("project manager")!= -1 or (job[0].find("project")!= -1 and job[0].replace("management","manager").find("manager")!= -1) or job[0].find("project coordinator")!= -1 or job[0].find("project consultant")!= -1 or (
                            job[0].find("manager")!= -1 ):
                            f.write("{} \t|\t project manager or equivalent\n".format(job[0].replace("|",",")))
                            label="project manager or equivalent"
                        else:
                            if job[0].find("leader")!= -1 or job[0].find("lead")!= -1:
                                f.write("{} \t|\t project leader or equivalent\n".format(job[0].replace("|",",")))
                                label="project leader or equivalent"
                            else:
                                if job[0].replace("sr","senior").find("senior")!= -1 or (job[0].find("technology")!= -1 and job[0].find("analyst")!= -1):
                                    f.write("{} \t|\t senior consultant or equivalent\n".format(job[0].replace("|",",")))
                                    label="senior consultant or equivalent"
                                else:
                                    if job[0].find("develop")!= -1 or job[0].find("system")!= -1 or job[0].find("tech")!= -1 or  job[0].find("analyst")!= -1 or  job[0].find("software")!= -1 or  job[0].find("it")!= -1 or  job[0].find("oracle")!= -1 or  job[0].find("consultant")!= -1 or  job[0].replace("jr","junior").find("junior")!= -1 or  job[0].find("engineer")!= -1:
                                        f.write("{} \t|\t it consultant or equivalent\n".format(job[0].replace("|",",")))
                                        label="it consultant or equivalent"
                                    else:
                                        f.write("{} \t|\t OOB\n".format(job[0].replace("|",",")))
                                        label="OOB"

        tnkstr= tnzr.word_tokenize(job[1].replace("\\n"," ").lower())
        jobs_labeled.append([label,job[0],[word for word in tnkstr if (word not in stoplist)]])
        #TaggedDocument creation 
        documents.append(TaggedDocument([word for word in tnkstr if (word not in stoplist)],[label.replace(" ","_")]))            
    f.close()
    print("Archivo temporal creado en: {}".format(os.getcwd()))
    return documents,jobs_labeled

In [6]:
fname= 'C:\\Users\\Victor\\Documents\\Compartido/LinkedinCV(300).json'
temp_file='JobsLabeled.txt'

resumes = open_file(fname)
job_desc = create_arr_by_job(resumes)
[documents,jobs_labeled] = label_jobs(temp_file,job_desc)

Total de CV:234
Total de puestos por experiencia:1340
Total de puestos extraidos:1574
Archivo temporal creado en: C:\Users\Victor\Documents\Compartido\LinkedInCV\Clasificacion


In [91]:
# Definición del modelo 
model = models.Doc2Vec(documents, dm = 0, alpha=0.025, vector_size= 200,window=10, min_alpha=0.025, min_count=1)

2018-11-30 13:48:25,468 : INFO : collecting all words and their counts
2018-11-30 13:48:25,469 : INFO : PROGRESS: at example #0, processed 0 words (0/s), 0 word types, 0 tags
2018-11-30 13:48:25,493 : INFO : collected 11394 word types and 9 unique tags from a corpus of 1574 examples and 93817 words
2018-11-30 13:48:25,497 : INFO : Loading a fresh vocabulary
2018-11-30 13:48:25,521 : INFO : effective_min_count=1 retains 11394 unique words (100% of original 11394, drops 0)
2018-11-30 13:48:25,522 : INFO : effective_min_count=1 leaves 93817 word corpus (100% of original 93817, drops 0)
2018-11-30 13:48:25,560 : INFO : deleting the raw counts dictionary of 11394 items
2018-11-30 13:48:25,560 : INFO : sample=0.001 downsamples 42 most-common words
2018-11-30 13:48:25,560 : INFO : downsampling leaves estimated 88291 word corpus (94.1% of prior 93817)
2018-11-30 13:48:25,593 : INFO : estimated required memory for 11394 words and 200 dimensions: 23936400 bytes
2018-11-30 13:48:25,595 : INFO : r

In [97]:
#Training
model.train(documents,total_examples=len(documents),epochs=200)

2018-11-30 13:54:01,266 : INFO : training model with 3 workers on 11394 vocabulary and 200 features, using sg=1 hs=0 sample=0.001 negative=5 window=10
2018-11-30 13:54:01,351 : INFO : worker thread finished; awaiting finish of 2 more threads
2018-11-30 13:54:01,362 : INFO : worker thread finished; awaiting finish of 1 more threads
2018-11-30 13:54:01,367 : INFO : worker thread finished; awaiting finish of 0 more threads
2018-11-30 13:54:01,367 : INFO : EPOCH - 1 : training on 93817 raw words (89859 effective words) took 0.1s, 921164 effective words/s
2018-11-30 13:54:01,450 : INFO : worker thread finished; awaiting finish of 2 more threads
2018-11-30 13:54:01,453 : INFO : worker thread finished; awaiting finish of 1 more threads
2018-11-30 13:54:01,457 : INFO : worker thread finished; awaiting finish of 0 more threads
2018-11-30 13:54:01,458 : INFO : EPOCH - 2 : training on 93817 raw words (89911 effective words) took 0.1s, 1027080 effective words/s
2018-11-30 13:54:01,528 : INFO : wor

2018-11-30 13:54:03,250 : INFO : worker thread finished; awaiting finish of 2 more threads
2018-11-30 13:54:03,267 : INFO : worker thread finished; awaiting finish of 1 more threads
2018-11-30 13:54:03,272 : INFO : worker thread finished; awaiting finish of 0 more threads
2018-11-30 13:54:03,272 : INFO : EPOCH - 21 : training on 93817 raw words (89915 effective words) took 0.1s, 987974 effective words/s
2018-11-30 13:54:03,349 : INFO : worker thread finished; awaiting finish of 2 more threads
2018-11-30 13:54:03,363 : INFO : worker thread finished; awaiting finish of 1 more threads
2018-11-30 13:54:03,369 : INFO : worker thread finished; awaiting finish of 0 more threads
2018-11-30 13:54:03,370 : INFO : EPOCH - 22 : training on 93817 raw words (89822 effective words) took 0.1s, 956168 effective words/s
2018-11-30 13:54:03,450 : INFO : worker thread finished; awaiting finish of 2 more threads
2018-11-30 13:54:03,457 : INFO : worker thread finished; awaiting finish of 1 more threads
2018

2018-11-30 13:54:05,105 : INFO : worker thread finished; awaiting finish of 1 more threads
2018-11-30 13:54:05,109 : INFO : worker thread finished; awaiting finish of 0 more threads
2018-11-30 13:54:05,110 : INFO : EPOCH - 41 : training on 93817 raw words (89938 effective words) took 0.1s, 1026875 effective words/s
2018-11-30 13:54:05,182 : INFO : worker thread finished; awaiting finish of 2 more threads
2018-11-30 13:54:05,198 : INFO : worker thread finished; awaiting finish of 1 more threads
2018-11-30 13:54:05,202 : INFO : worker thread finished; awaiting finish of 0 more threads
2018-11-30 13:54:05,203 : INFO : EPOCH - 42 : training on 93817 raw words (89894 effective words) took 0.1s, 1014664 effective words/s
2018-11-30 13:54:05,280 : INFO : worker thread finished; awaiting finish of 2 more threads
2018-11-30 13:54:05,287 : INFO : worker thread finished; awaiting finish of 1 more threads
2018-11-30 13:54:05,292 : INFO : worker thread finished; awaiting finish of 0 more threads
20

2018-11-30 13:54:06,925 : INFO : worker thread finished; awaiting finish of 0 more threads
2018-11-30 13:54:06,926 : INFO : EPOCH - 61 : training on 93817 raw words (89859 effective words) took 0.1s, 1010015 effective words/s
2018-11-30 13:54:06,996 : INFO : worker thread finished; awaiting finish of 2 more threads
2018-11-30 13:54:07,012 : INFO : worker thread finished; awaiting finish of 1 more threads
2018-11-30 13:54:07,016 : INFO : worker thread finished; awaiting finish of 0 more threads
2018-11-30 13:54:07,017 : INFO : EPOCH - 62 : training on 93817 raw words (89908 effective words) took 0.1s, 1023686 effective words/s
2018-11-30 13:54:07,098 : INFO : worker thread finished; awaiting finish of 2 more threads
2018-11-30 13:54:07,105 : INFO : worker thread finished; awaiting finish of 1 more threads
2018-11-30 13:54:07,111 : INFO : worker thread finished; awaiting finish of 0 more threads
2018-11-30 13:54:07,111 : INFO : EPOCH - 63 : training on 93817 raw words (89770 effective wo

2018-11-30 13:54:08,746 : INFO : EPOCH - 81 : training on 93817 raw words (89843 effective words) took 0.1s, 1031031 effective words/s
2018-11-30 13:54:08,818 : INFO : worker thread finished; awaiting finish of 2 more threads
2018-11-30 13:54:08,832 : INFO : worker thread finished; awaiting finish of 1 more threads
2018-11-30 13:54:08,837 : INFO : worker thread finished; awaiting finish of 0 more threads
2018-11-30 13:54:08,838 : INFO : EPOCH - 82 : training on 93817 raw words (89903 effective words) took 0.1s, 1017867 effective words/s
2018-11-30 13:54:08,915 : INFO : worker thread finished; awaiting finish of 2 more threads
2018-11-30 13:54:08,922 : INFO : worker thread finished; awaiting finish of 1 more threads
2018-11-30 13:54:08,928 : INFO : worker thread finished; awaiting finish of 0 more threads
2018-11-30 13:54:08,929 : INFO : EPOCH - 83 : training on 93817 raw words (89761 effective words) took 0.1s, 1022172 effective words/s
2018-11-30 13:54:08,997 : INFO : worker thread fi

2018-11-30 13:54:10,629 : INFO : worker thread finished; awaiting finish of 2 more threads
2018-11-30 13:54:10,640 : INFO : worker thread finished; awaiting finish of 1 more threads
2018-11-30 13:54:10,645 : INFO : worker thread finished; awaiting finish of 0 more threads
2018-11-30 13:54:10,646 : INFO : EPOCH - 102 : training on 93817 raw words (89970 effective words) took 0.1s, 1019578 effective words/s
2018-11-30 13:54:10,714 : INFO : worker thread finished; awaiting finish of 2 more threads
2018-11-30 13:54:10,732 : INFO : worker thread finished; awaiting finish of 1 more threads
2018-11-30 13:54:10,737 : INFO : worker thread finished; awaiting finish of 0 more threads
2018-11-30 13:54:10,737 : INFO : EPOCH - 103 : training on 93817 raw words (89918 effective words) took 0.1s, 1036462 effective words/s
2018-11-30 13:54:10,814 : INFO : worker thread finished; awaiting finish of 2 more threads
2018-11-30 13:54:10,822 : INFO : worker thread finished; awaiting finish of 1 more threads


2018-11-30 13:54:12,447 : INFO : worker thread finished; awaiting finish of 1 more threads
2018-11-30 13:54:12,450 : INFO : worker thread finished; awaiting finish of 0 more threads
2018-11-30 13:54:12,451 : INFO : EPOCH - 122 : training on 93817 raw words (89867 effective words) took 0.1s, 1036459 effective words/s
2018-11-30 13:54:12,529 : INFO : worker thread finished; awaiting finish of 2 more threads
2018-11-30 13:54:12,536 : INFO : worker thread finished; awaiting finish of 1 more threads
2018-11-30 13:54:12,541 : INFO : worker thread finished; awaiting finish of 0 more threads
2018-11-30 13:54:12,542 : INFO : EPOCH - 123 : training on 93817 raw words (89860 effective words) took 0.1s, 1027660 effective words/s
2018-11-30 13:54:12,612 : INFO : worker thread finished; awaiting finish of 2 more threads
2018-11-30 13:54:12,625 : INFO : worker thread finished; awaiting finish of 1 more threads
2018-11-30 13:54:12,629 : INFO : worker thread finished; awaiting finish of 0 more threads


2018-11-30 13:54:14,275 : INFO : worker thread finished; awaiting finish of 0 more threads
2018-11-30 13:54:14,276 : INFO : EPOCH - 142 : training on 93817 raw words (89888 effective words) took 0.1s, 991278 effective words/s
2018-11-30 13:54:14,359 : INFO : worker thread finished; awaiting finish of 2 more threads
2018-11-30 13:54:14,361 : INFO : worker thread finished; awaiting finish of 1 more threads
2018-11-30 13:54:14,365 : INFO : worker thread finished; awaiting finish of 0 more threads
2018-11-30 13:54:14,366 : INFO : EPOCH - 143 : training on 93817 raw words (89879 effective words) took 0.1s, 1029132 effective words/s
2018-11-30 13:54:14,447 : INFO : worker thread finished; awaiting finish of 2 more threads
2018-11-30 13:54:14,453 : INFO : worker thread finished; awaiting finish of 1 more threads
2018-11-30 13:54:14,458 : INFO : worker thread finished; awaiting finish of 0 more threads
2018-11-30 13:54:14,458 : INFO : EPOCH - 144 : training on 93817 raw words (89894 effective 

2018-11-30 13:54:16,086 : INFO : EPOCH - 162 : training on 93817 raw words (89870 effective words) took 0.1s, 1044152 effective words/s
2018-11-30 13:54:16,163 : INFO : worker thread finished; awaiting finish of 2 more threads
2018-11-30 13:54:16,163 : INFO : worker thread finished; awaiting finish of 1 more threads
2018-11-30 13:54:16,176 : INFO : worker thread finished; awaiting finish of 0 more threads
2018-11-30 13:54:16,177 : INFO : EPOCH - 163 : training on 93817 raw words (89754 effective words) took 0.1s, 1013325 effective words/s
2018-11-30 13:54:16,261 : INFO : worker thread finished; awaiting finish of 2 more threads
2018-11-30 13:54:16,267 : INFO : worker thread finished; awaiting finish of 1 more threads
2018-11-30 13:54:16,272 : INFO : worker thread finished; awaiting finish of 0 more threads
2018-11-30 13:54:16,273 : INFO : EPOCH - 164 : training on 93817 raw words (89855 effective words) took 0.1s, 993575 effective words/s
2018-11-30 13:54:16,345 : INFO : worker thread 

2018-11-30 13:54:18,036 : INFO : worker thread finished; awaiting finish of 2 more threads
2018-11-30 13:54:18,039 : INFO : worker thread finished; awaiting finish of 1 more threads
2018-11-30 13:54:18,046 : INFO : worker thread finished; awaiting finish of 0 more threads
2018-11-30 13:54:18,047 : INFO : EPOCH - 183 : training on 93817 raw words (89898 effective words) took 0.1s, 822295 effective words/s
2018-11-30 13:54:18,130 : INFO : worker thread finished; awaiting finish of 2 more threads
2018-11-30 13:54:18,135 : INFO : worker thread finished; awaiting finish of 1 more threads
2018-11-30 13:54:18,140 : INFO : worker thread finished; awaiting finish of 0 more threads
2018-11-30 13:54:18,141 : INFO : EPOCH - 184 : training on 93817 raw words (89905 effective words) took 0.1s, 994450 effective words/s
2018-11-30 13:54:18,234 : INFO : worker thread finished; awaiting finish of 2 more threads
2018-11-30 13:54:18,241 : INFO : worker thread finished; awaiting finish of 1 more threads
20

In [105]:
model.save("Doc2VecbyJob.model")
jobs_labeled[25][0:2]

2018-11-30 14:38:49,843 : INFO : saving Doc2Vec object under Doc2VecbyJob.model, separately None
2018-11-30 14:38:50,145 : INFO : saved Doc2VecbyJob.model


['project leader or equivalent', 'technical leader']

In [5]:
m = Doc2Vec.load("Doc2VecbyJob.model")

lbl=[]
emb=[]

for job in jobs_labeled:
    m.random.seed(0)
    vecjob = m.infer_vector(doc_words=job[2])#, steps=20, alpha=0.025)
    lbl.append(job[0:2])
    emb.append(vecjob)

print(len(lbl))
print(len(emb))

2019-08-20 15:21:30,104 : INFO : loading Doc2Vec object from Doc2VecbyJob.model
2019-08-20 15:21:30,246 : INFO : loading docvecs recursively from Doc2VecbyJob.model.docvecs.* with mmap=None
2019-08-20 15:21:30,247 : INFO : loading trainables recursively from Doc2VecbyJob.model.trainables.* with mmap=None
2019-08-20 15:21:30,247 : INFO : loading wv recursively from Doc2VecbyJob.model.wv.* with mmap=None
2019-08-20 15:21:30,248 : INFO : loading vocabulary recursively from Doc2VecbyJob.model.vocabulary.* with mmap=None
2019-08-20 15:21:30,248 : INFO : loaded Doc2VecbyJob.model


1574
1574


In [6]:
docs_emb = {"Labels": lbl
           ,"Embedding": np.array(emb)}
docs_emb["Embedding"]

print ("{}".format(docs_emb["Labels"][3]))
print ("{}".format(docs_emb["Embedding"][3]))



['project manager or equivalent', 'project manager']
[ 3.96857053e-01  1.05057466e+00  7.26938695e-02  2.49999747e-01
 -9.20711160e-02 -8.84340465e-01  3.86154741e-01 -2.85153437e+00
  2.56532401e-01  1.41724861e+00  4.98221487e-01 -8.35338295e-01
  4.19762023e-02 -5.76262288e-02 -3.38163316e-01  6.05112135e-01
  5.53938746e-01 -5.74627340e-01  5.41164503e-02 -8.96303430e-02
 -5.93435228e-01 -5.87558568e-01  4.00690362e-02  8.41903865e-01
  3.93427223e-01  1.21233039e-01  3.69861513e-01 -4.18008864e-01
 -1.84263587e-01  2.47164890e-01  3.67779993e-02  2.20003072e-02
 -1.34493724e-01 -1.53937638e-02  2.38979995e-01 -4.69090849e-01
 -5.33349693e-01  8.67960155e-01  1.18465245e-01 -8.63864422e-01
 -3.24613482e-01  2.31685936e-01 -3.78684819e-01 -4.64763016e-01
 -6.50612488e-02  2.97890365e-01  6.37624562e-02 -6.37008071e-01
  3.76300931e-01  3.21452081e-01 -1.46881863e-01 -2.84171160e-02
 -2.36480862e-01 -4.34510946e-01 -5.53270057e-02 -2.36849546e-01
 -1.18249863e-01  1.85100541e-01  1.3

In [13]:
def save_tensorboard_proj(loc,dict_emb,lbl_file):
    LOG_DIR = os.path.join(os.getcwd(),loc)
    EMBEDDING_SIZE = len(dict_emb["Embedding"][0])
    VOCA_SIZE = len(dict_emb["Labels"])
    LABEL_FILE = os.path.join(os.getcwd(), os.path.join(loc,lbl_file))
    print("The current working directory:", LOG_DIR,"\n")
    print("the current LABEL_FILE:", LABEL_FILE, "\n")
    Labels=dict_emb["Labels"] 

    #Write label file 
    with open(LABEL_FILE,"w") as f:
        f.write("Generic Job Name\t Real Job Name\n")
        for i in range(VOCA_SIZE):
            f.write("{}\t{}\n".format(Labels[i][0].encode("utf-8"),Labels[i][1].encode("utf-8")))
                           
    print("{} file Created\n".format(lbl_file))
                        
    # word vector of embedding. 
    embedding_input = tf.Variable(dict_emb["Embedding"], dtype=tf.float32, name="Embeddings")

    # For input of Tensorboard Projector
    embeddings = tf.Variable(tf.zeros([VOCA_SIZE,EMBEDDING_SIZE], name="embedding_intial_tensor"), name="real_Embedding")
    assignment = embeddings.assign(embedding_input) 
    
    # To initilize the variable. 
    init = tf.global_variables_initializer()
    saver = tf.train.Saver()
    
    with tf.Session() as sess:
        sess.run(init)
        
        # Format: tensorflow/contrib/tensorboard/plugins/projector/projector_config.proto
        config = tf.contrib.tensorboard.plugins.projector.ProjectorConfig()
        # You can add multiple embeddings. Here we add only one.
        embedding_config = config.embeddings.add()
        embedding_config.tensor_name = embedding_input.name
        # Link this tensor to its metadata file
        embedding_config.metadata_path = LABEL_FILE
        
        # Use the same LOG_DIR where you stored your checkpoint.
        summary_writer = tf.summary.FileWriter(LOG_DIR)
    
        # The next line writes a projector_config.pbtxt in the LOG_DtenIR. TensorBoard will
        # read this file during startup.
        tf.contrib.tensorboard.plugins.projector.visualize_embeddings(summary_writer, config)
    
        save_path1 = saver.save(sess, os.path.join(LOG_DIR, "Embedding_model.ckpt"), global_step=1)
        print("\nModel Saved in file: %s" % save_path1) 

In [16]:
Labels=docs_emb["Labels"] 
Labels

[['project manager or equivalent', 'project manager'],
 ['project manager or equivalent', 'project manager'],
 ['project manager or equivalent', 'project manager'],
 ['project manager or equivalent', 'project manager'],
 ['project manager or equivalent',
  'project manager/scrum master/business analyst'],
 ['project manager or equivalent', 'project manager'],
 ['vice president or equivalent', 'vice president, project management'],
 ['vice president or equivalent', 'vice president, product management'],
 ['vice president or equivalent', 'vice president'],
 ['vice president or equivalent', 'gm data platform & vp product'],
 ['project manager or equivalent',
  'it transition & transformation project manager'],
 ['project manager or equivalent', 'project manager'],
 ['project manager or equivalent',
  'scrum master/technical project manager - csm'],
 ['project manager or equivalent', 'data & analytics project manager'],
 ['project manager or equivalent', 'project manager'],
 ['project mana

In [8]:
loc = "logExperience/"
lbl_file = "lableExperience.tsv"
save_tensorboard_proj(loc,docs_emb,lbl_file)

The current working directory: C:\Users\Victor\Documents\Compartido\LinkedInCV\Clasificacion\logExperience/ 

the current LABEL_FILE: C:\Users\Victor\Documents\Compartido\LinkedInCV\Clasificacion\logExperience/lableExperience.tsv 

lableExperience.tsv file Created


Model Saved in file: C:\Users\Victor\Documents\Compartido\LinkedInCV\Clasificacion\logExperience/Embedding_model.ckpt-1


# By skills

In [7]:
def create_arr_by_skills(resumes):
    df=pd.DataFrame(resumes)
    job_desc=[]
    for cv in df.Resume:
        puesto=cv["job"].lower()
    #     print(puesto)
        skill=""
        for skills in cv['skills_endorsment']:
            skill=skill+" "+skills['skill_name'].replace(" ","_")
        skill=skill.lower()
    #     print(skill)
    #         cat=skills['skill_category']
        if len(skill)>0:
            job_desc.append([puesto,skill])
    return job_desc    

In [8]:
temp_file_s='JobsLabeled_s.txt'
job_desc_s = create_arr_by_skills(resumes)
[documents_s,jobs_labeled_s] = label_jobs(temp_file_s,job_desc_s)

Archivo temporal creado en: C:\Users\Victor\Documents\Compartido\LinkedInCV\Clasificacion


In [159]:
# Definición del modelo 
model_s = models.Doc2Vec(documents_s, dm = 0, alpha=0.015, vector_size= 200,window=10, min_alpha=0.025, min_count=1)

2018-11-30 15:31:33,687 : INFO : collecting all words and their counts
2018-11-30 15:31:33,689 : INFO : PROGRESS: at example #0, processed 0 words (0/s), 0 word types, 0 tags
2018-11-30 15:31:33,692 : INFO : collected 2454 word types and 8 unique tags from a corpus of 293 examples and 9848 words
2018-11-30 15:31:33,693 : INFO : Loading a fresh vocabulary
2018-11-30 15:31:33,699 : INFO : effective_min_count=1 retains 2454 unique words (100% of original 2454, drops 0)
2018-11-30 15:31:33,700 : INFO : effective_min_count=1 leaves 9848 word corpus (100% of original 9848, drops 0)
2018-11-30 15:31:33,707 : INFO : deleting the raw counts dictionary of 2454 items
2018-11-30 15:31:33,708 : INFO : sample=0.001 downsamples 73 most-common words
2018-11-30 15:31:33,709 : INFO : downsampling leaves estimated 8332 word corpus (84.6% of prior 9848)
2018-11-30 15:31:33,716 : INFO : estimated required memory for 2454 words and 200 dimensions: 5161400 bytes
2018-11-30 15:31:33,717 : INFO : resetting lay

In [160]:
#Training
model_s.train(documents_s,total_examples=len(documents_s),epochs=200)

2018-11-30 15:31:38,020 : INFO : training model with 3 workers on 2454 vocabulary and 200 features, using sg=1 hs=0 sample=0.001 negative=5 window=10
2018-11-30 15:31:38,025 : INFO : worker thread finished; awaiting finish of 2 more threads
2018-11-30 15:31:38,026 : INFO : worker thread finished; awaiting finish of 1 more threads
2018-11-30 15:31:38,052 : INFO : worker thread finished; awaiting finish of 0 more threads
2018-11-30 15:31:38,053 : INFO : EPOCH - 1 : training on 9848 raw words (8601 effective words) took 0.0s, 298896 effective words/s
2018-11-30 15:31:38,056 : INFO : worker thread finished; awaiting finish of 2 more threads
2018-11-30 15:31:38,057 : INFO : worker thread finished; awaiting finish of 1 more threads
2018-11-30 15:31:38,079 : INFO : worker thread finished; awaiting finish of 0 more threads
2018-11-30 15:31:38,080 : INFO : EPOCH - 2 : training on 9848 raw words (8635 effective words) took 0.0s, 352505 effective words/s
2018-11-30 15:31:38,086 : INFO : worker th

2018-11-30 15:31:38,587 : INFO : worker thread finished; awaiting finish of 2 more threads
2018-11-30 15:31:38,588 : INFO : worker thread finished; awaiting finish of 1 more threads
2018-11-30 15:31:38,610 : INFO : worker thread finished; awaiting finish of 0 more threads
2018-11-30 15:31:38,611 : INFO : EPOCH - 21 : training on 9848 raw words (8594 effective words) took 0.0s, 346335 effective words/s
2018-11-30 15:31:38,614 : INFO : worker thread finished; awaiting finish of 2 more threads
2018-11-30 15:31:38,616 : INFO : worker thread finished; awaiting finish of 1 more threads
2018-11-30 15:31:38,642 : INFO : worker thread finished; awaiting finish of 0 more threads
2018-11-30 15:31:38,644 : INFO : EPOCH - 22 : training on 9848 raw words (8636 effective words) took 0.0s, 283922 effective words/s
2018-11-30 15:31:38,648 : INFO : worker thread finished; awaiting finish of 2 more threads
2018-11-30 15:31:38,649 : INFO : worker thread finished; awaiting finish of 1 more threads
2018-11-

2018-11-30 15:31:39,213 : INFO : worker thread finished; awaiting finish of 0 more threads
2018-11-30 15:31:39,214 : INFO : EPOCH - 41 : training on 9848 raw words (8681 effective words) took 0.0s, 371599 effective words/s
2018-11-30 15:31:39,220 : INFO : worker thread finished; awaiting finish of 2 more threads
2018-11-30 15:31:39,221 : INFO : worker thread finished; awaiting finish of 1 more threads
2018-11-30 15:31:39,245 : INFO : worker thread finished; awaiting finish of 0 more threads
2018-11-30 15:31:39,246 : INFO : EPOCH - 42 : training on 9848 raw words (8593 effective words) took 0.0s, 322175 effective words/s
2018-11-30 15:31:39,250 : INFO : worker thread finished; awaiting finish of 2 more threads
2018-11-30 15:31:39,252 : INFO : worker thread finished; awaiting finish of 1 more threads
2018-11-30 15:31:39,275 : INFO : worker thread finished; awaiting finish of 0 more threads
2018-11-30 15:31:39,276 : INFO : EPOCH - 43 : training on 9848 raw words (8621 effective words) too

2018-11-30 15:31:39,769 : INFO : worker thread finished; awaiting finish of 2 more threads
2018-11-30 15:31:39,770 : INFO : worker thread finished; awaiting finish of 1 more threads
2018-11-30 15:31:39,791 : INFO : worker thread finished; awaiting finish of 0 more threads
2018-11-30 15:31:39,793 : INFO : EPOCH - 62 : training on 9848 raw words (8564 effective words) took 0.0s, 343058 effective words/s
2018-11-30 15:31:39,797 : INFO : worker thread finished; awaiting finish of 2 more threads
2018-11-30 15:31:39,798 : INFO : worker thread finished; awaiting finish of 1 more threads
2018-11-30 15:31:39,826 : INFO : worker thread finished; awaiting finish of 0 more threads
2018-11-30 15:31:39,827 : INFO : EPOCH - 63 : training on 9848 raw words (8633 effective words) took 0.0s, 284643 effective words/s
2018-11-30 15:31:39,832 : INFO : worker thread finished; awaiting finish of 2 more threads
2018-11-30 15:31:39,833 : INFO : worker thread finished; awaiting finish of 1 more threads
2018-11-

2018-11-30 15:31:40,407 : INFO : worker thread finished; awaiting finish of 0 more threads
2018-11-30 15:31:40,408 : INFO : EPOCH - 82 : training on 9848 raw words (8658 effective words) took 0.0s, 269448 effective words/s
2018-11-30 15:31:40,413 : INFO : worker thread finished; awaiting finish of 2 more threads
2018-11-30 15:31:40,414 : INFO : worker thread finished; awaiting finish of 1 more threads
2018-11-30 15:31:40,439 : INFO : worker thread finished; awaiting finish of 0 more threads
2018-11-30 15:31:40,440 : INFO : EPOCH - 83 : training on 9848 raw words (8590 effective words) took 0.0s, 303634 effective words/s
2018-11-30 15:31:40,446 : INFO : worker thread finished; awaiting finish of 2 more threads
2018-11-30 15:31:40,447 : INFO : worker thread finished; awaiting finish of 1 more threads
2018-11-30 15:31:40,476 : INFO : worker thread finished; awaiting finish of 0 more threads
2018-11-30 15:31:40,477 : INFO : EPOCH - 84 : training on 9848 raw words (8619 effective words) too

2018-11-30 15:31:41,042 : INFO : EPOCH - 102 : training on 9848 raw words (8616 effective words) took 0.0s, 288342 effective words/s
2018-11-30 15:31:41,046 : INFO : worker thread finished; awaiting finish of 2 more threads
2018-11-30 15:31:41,047 : INFO : worker thread finished; awaiting finish of 1 more threads
2018-11-30 15:31:41,075 : INFO : worker thread finished; awaiting finish of 0 more threads
2018-11-30 15:31:41,075 : INFO : EPOCH - 103 : training on 9848 raw words (8627 effective words) took 0.0s, 284782 effective words/s
2018-11-30 15:31:41,080 : INFO : worker thread finished; awaiting finish of 2 more threads
2018-11-30 15:31:41,081 : INFO : worker thread finished; awaiting finish of 1 more threads
2018-11-30 15:31:41,106 : INFO : worker thread finished; awaiting finish of 0 more threads
2018-11-30 15:31:41,107 : INFO : EPOCH - 104 : training on 9848 raw words (8616 effective words) took 0.0s, 318242 effective words/s
2018-11-30 15:31:41,111 : INFO : worker thread finished

2018-11-30 15:31:41,587 : INFO : worker thread finished; awaiting finish of 2 more threads
2018-11-30 15:31:41,588 : INFO : worker thread finished; awaiting finish of 1 more threads
2018-11-30 15:31:41,612 : INFO : worker thread finished; awaiting finish of 0 more threads
2018-11-30 15:31:41,613 : INFO : EPOCH - 123 : training on 9848 raw words (8552 effective words) took 0.0s, 319032 effective words/s
2018-11-30 15:31:41,616 : INFO : worker thread finished; awaiting finish of 2 more threads
2018-11-30 15:31:41,617 : INFO : worker thread finished; awaiting finish of 1 more threads
2018-11-30 15:31:41,638 : INFO : worker thread finished; awaiting finish of 0 more threads
2018-11-30 15:31:41,639 : INFO : EPOCH - 124 : training on 9848 raw words (8625 effective words) took 0.0s, 357738 effective words/s
2018-11-30 15:31:41,642 : INFO : worker thread finished; awaiting finish of 2 more threads
2018-11-30 15:31:41,643 : INFO : worker thread finished; awaiting finish of 1 more threads
2018-1

2018-11-30 15:31:42,158 : INFO : worker thread finished; awaiting finish of 1 more threads
2018-11-30 15:31:42,182 : INFO : worker thread finished; awaiting finish of 0 more threads
2018-11-30 15:31:42,182 : INFO : EPOCH - 143 : training on 9848 raw words (8599 effective words) took 0.0s, 324525 effective words/s
2018-11-30 15:31:42,189 : INFO : worker thread finished; awaiting finish of 2 more threads
2018-11-30 15:31:42,190 : INFO : worker thread finished; awaiting finish of 1 more threads
2018-11-30 15:31:42,210 : INFO : worker thread finished; awaiting finish of 0 more threads
2018-11-30 15:31:42,211 : INFO : EPOCH - 144 : training on 9848 raw words (8676 effective words) took 0.0s, 380375 effective words/s
2018-11-30 15:31:42,214 : INFO : worker thread finished; awaiting finish of 2 more threads
2018-11-30 15:31:42,215 : INFO : worker thread finished; awaiting finish of 1 more threads
2018-11-30 15:31:42,237 : INFO : worker thread finished; awaiting finish of 0 more threads
2018-1

2018-11-30 15:31:42,721 : INFO : worker thread finished; awaiting finish of 0 more threads
2018-11-30 15:31:42,722 : INFO : EPOCH - 163 : training on 9848 raw words (8616 effective words) took 0.0s, 268065 effective words/s
2018-11-30 15:31:42,725 : INFO : worker thread finished; awaiting finish of 2 more threads
2018-11-30 15:31:42,726 : INFO : worker thread finished; awaiting finish of 1 more threads
2018-11-30 15:31:42,747 : INFO : worker thread finished; awaiting finish of 0 more threads
2018-11-30 15:31:42,748 : INFO : EPOCH - 164 : training on 9848 raw words (8627 effective words) took 0.0s, 367511 effective words/s
2018-11-30 15:31:42,752 : INFO : worker thread finished; awaiting finish of 2 more threads
2018-11-30 15:31:42,753 : INFO : worker thread finished; awaiting finish of 1 more threads
2018-11-30 15:31:42,771 : INFO : worker thread finished; awaiting finish of 0 more threads
2018-11-30 15:31:42,772 : INFO : EPOCH - 165 : training on 9848 raw words (8640 effective words) 

2018-11-30 15:31:43,251 : INFO : EPOCH - 183 : training on 9848 raw words (8607 effective words) took 0.0s, 334730 effective words/s
2018-11-30 15:31:43,255 : INFO : worker thread finished; awaiting finish of 2 more threads
2018-11-30 15:31:43,255 : INFO : worker thread finished; awaiting finish of 1 more threads
2018-11-30 15:31:43,274 : INFO : worker thread finished; awaiting finish of 0 more threads
2018-11-30 15:31:43,275 : INFO : EPOCH - 184 : training on 9848 raw words (8608 effective words) took 0.0s, 404259 effective words/s
2018-11-30 15:31:43,278 : INFO : worker thread finished; awaiting finish of 2 more threads
2018-11-30 15:31:43,279 : INFO : worker thread finished; awaiting finish of 1 more threads
2018-11-30 15:31:43,296 : INFO : worker thread finished; awaiting finish of 0 more threads
2018-11-30 15:31:43,296 : INFO : EPOCH - 185 : training on 9848 raw words (8562 effective words) took 0.0s, 440378 effective words/s
2018-11-30 15:31:43,301 : INFO : worker thread finished

In [161]:
model_s.save("Doc2VecbySkill.model")
jobs_labeled[25][0:2]

2018-11-30 15:31:53,493 : INFO : saving Doc2Vec object under Doc2VecbySkill.model, separately None
2018-11-30 15:31:53,545 : INFO : saved Doc2VecbySkill.model


['project leader or equivalent', 'technical leader']

In [10]:
ms = Doc2Vec.load("Doc2VecbySkill.model")

lbl_s=[]
emb_s=[]

for job in jobs_labeled_s:
    ms.random.seed(0)
    vecjob = ms.infer_vector(doc_words=job[2])#, steps=20, alpha=0.025)
    lbl_s.append(job[0:2])
    emb_s.append(vecjob)

print(len(lbl_s))
print(len(emb_s))

2019-08-20 15:32:20,497 : INFO : loading Doc2Vec object from Doc2VecbySkill.model
2019-08-20 15:32:20,530 : INFO : loading docvecs recursively from Doc2VecbySkill.model.docvecs.* with mmap=None
2019-08-20 15:32:20,531 : INFO : loading trainables recursively from Doc2VecbySkill.model.trainables.* with mmap=None
2019-08-20 15:32:20,531 : INFO : loading wv recursively from Doc2VecbySkill.model.wv.* with mmap=None
2019-08-20 15:32:20,532 : INFO : loading vocabulary recursively from Doc2VecbySkill.model.vocabulary.* with mmap=None
2019-08-20 15:32:20,533 : INFO : loaded Doc2VecbySkill.model


293
293


In [11]:
docs_emb = {"Labels": lbl_s
           ,"Embedding": np.array(emb_s)}
docs_emb["Embedding"]


array([[ 0.21242899,  0.25383812, -0.2640538 , ...,  0.5132216 ,
         0.04010211,  0.22781047],
       [-0.00716305, -0.3011497 ,  0.23457043, ...,  0.00244293,
        -0.46173203, -0.5839876 ],
       [ 0.5331173 , -0.12258284,  0.01947141, ..., -0.38452208,
         0.42872673, -0.53685296],
       ...,
       [ 0.83014697, -0.08754422,  0.31440774, ..., -0.4186766 ,
         0.15213071, -0.4213813 ],
       [ 0.62552434,  0.06785432,  0.09709484, ..., -0.36457658,
         0.338376  , -0.43566465],
       [ 0.4127543 ,  0.01102953, -0.04850457, ..., -0.24391784,
         0.18425617, -0.58491063]], dtype=float32)

In [14]:
loc = "logSkills/"
lbl_file = "lableSkills.tsv"
save_tensorboard_proj(loc,docs_emb,lbl_file)

The current working directory: C:\Users\Victor\Documents\Compartido\LinkedInCV\Clasificacion\logSkills/ 

the current LABEL_FILE: C:\Users\Victor\Documents\Compartido\LinkedInCV\Clasificacion\logSkills/lableSkills.tsv 

lableSkills.tsv file Created


Model Saved in file: C:\Users\Victor\Documents\Compartido\LinkedInCV\Clasificacion\logSkills/Embedding_model.ckpt-1
